In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()
df

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл..."
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ..."
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак..."
...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,Заявка на транзит российского газа через Украи...
168731,168731,2024-12-31 19:45:52,2024,12,31,\n\n▪️Российские войска нанесли групповой удар...
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, ..."
168733,168733,2024-12-31 21:05:02,2024,12,31,Повторные выбросы нефтепродуктов зафиксированы...


In [3]:
# df = df.drop(columns=['multi_labels', 'hier_label']).copy()

In [4]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="cuda",
#     torch_dtype="auto",
# )

In [5]:
# def generate_categories(news_text):
#     prompt = f"""
# Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

# - **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
# - **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

# Входные данные - словарь (dict) с ключами: 'text' (текст новости).

# Текст новости: {news_text}

# Ответ:
# """
#     model.config.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

#     input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
#     outputs = model.generate(**input_ids, max_length=2048)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print(response)

#     # Извлекаем JSON из текста
#     json_match = re.search(r"\{.*\}", response, re.DOTALL)
#     if json_match:
#         json_text = json_match.group(0)
#         try:
#             return json.loads(json_text)
#         except json.JSONDecodeError:
#             return {"multi_labels": [], "hier_label": []}  # Возвращаем пустую структуру при ошибке
#     else:
#         return {"multi_labels": [], "hier_label": []}


# def process_first_n_rows(df, n):
#     """Обрабатывает первые n строк DataFrame, добавляя категории."""

#     # Создаем копию первых n строк, чтобы избежать SettingWithCopyWarning
#     df_subset = df.head(n).copy()

#     # Применяем функцию generate_categories к подмножеству DataFrame
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)

#     return df_subset

# N = 10
# df_processed = process_first_n_rows(df, N)

# df = df.drop(index=df.index[:N])  #удаляем старые N строк, которые мы обработали
# df = pd.concat([df_processed, df]).reset_index(drop=True) #объединяем обработанные строки с остальными

# df

In [6]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# # --- Модифицированный код для модели T-lite ---
# MODEL_NAME = "t-tech/T-lite-it-1.0"

# # Инициализация модели и токенизатора
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="auto",
#     torch_dtype="auto",
# )

# def generate_categories(news_text):
#     # Формируем системный промпт и запрос пользователя
#     messages = [
#         {
#             "role": "system",
#             "content": """Ты — редактор новостного агрегатора в России. Твоя задача: категоризировать новости для российских читателей, создав две колонки: multi_labels и hier_label.
            
#             multi_labels: Список [,] из наиболее значимых категорий новости.
#             hier_label: Список [,] - Двухуровневая иерархия для первой категории.
            
#             Необходим ответ без пояснений строго в формате JSON."""
#         },
#         {
#             "role": "user", 
#             "content": f"Текст новости: {news_text}"
#         }
#     ]
    
#     # Применяем чатовый шаблон
#     text = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )
    
#     # Токенизация и генерация
#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
#     generated_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=128,
#         pad_token_id=tokenizer.eos_token_id
#     )
    
#     # Извлекаем сгенерированный текст
#     generated_ids = [
#         output_ids[len(input_ids):] 
#         for input_ids, output_ids 
#         in zip(model_inputs.input_ids, generated_ids)
#     ]
#     response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print(response)
#     # Парсинг JSON
#     try:
#         json_match = re.search(r"\{.*\}", response, re.DOTALL)
#         return json.loads(json_match.group(0)) if json_match else {"multi_labels": [], "hier_label": []}
#     except json.JSONDecodeError:
#         return {"multi_labels": [], "hier_label": []}

# # Остальной код обработки DataFrame остается без изменений
# def process_first_n_rows(df, n):
#     df_subset = df.head(n).copy()
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)
#     return df_subset

# # Параметры обработки
# N = 10  # Сколько строк обрабатывать за раз

# # Основной цикл обработки
# df_processed = process_first_n_rows(df, N)
# print("Обработанные данные:")
# print(df_processed)

# # Обновление основного DataFrame (как в твоем оригинальном коде)
# df = df.drop(index=df.index[:N])
# df = pd.concat([df_processed, df]).reset_index(drop=True)
# df

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df.head(10)

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, если кто-то из американцев в Ираке пострадает от нападений: «Иран понесет полную ответственность за гибель людей или ущерб, причиненный какому-либо из наших объектов. Они заплатят очень большую цену! Это не предупреждение, а угроза. Счастливого нового года!»\n\nРанее протестующие проникли за внешний периметр ограды посольства США в Багдаде. США обвинили в нападении власти Ирана, Тегеран это отрицает."
1,1,2020-01-01 00:54:16,2020,1,1,"Ким Чен Ын пообещал скоро представить новое стратегическое оружие КНДР и заявил, что будет продолжать разработку стратегического оружия, пока враждебная политика США не будет устранена"
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили Новый год, участвуя в праздничных мероприятиях, сообщило МВД РФ. Нарушений правопорядка не допущено. В МВД добавили, что общественный порядок охраняли около 67,4 тысячи полицейских."
3,3,2020-01-01 12:55:30,2020,1,1,"Столкновения произошли в Гонконге во время согласованной акции, полиция применила слезоточивый газ для разгона протестующих, сообщает газета South China Morning Post."
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что накануне отшлепал по руке одну не в меру активную паломницу.\n\nВо вторник вечером женщина не на шутку рассердила папу Франциска, который вышел на площадь Святого Петра в Ватикане, чтобы полюбоваться установленными там под елью рождественскими яслями и поприветствовать верующих. Понтифик по обычаю пожимал присутствующим руки и уже собирался вернуться в папские покои, когда паломница неожиданно схватила его за правую руку и бесцеремонно потянула ее к себе.\n\n"
5,5,2020-01-01 15:45:51,2020,1,1,"Более 75,5 тысяч пассажиров встретили Новый год в поездах дальнего следования, сообщила ""Федеральная пассажирская компания"""
6,6,2020-01-01 17:12:14,2020,1,1,"В России в ночь на 1 января , сообщает «Столото». Теперь счастливчика ждут для оформления выигрыша. Пока известно лишь то, что билет был куплен в Москве."
7,7,2020-01-01 19:38:39,2020,1,1,"Украинские силовики заявили, что в новом году , ранен военный.\n\nНесмотря на то что ""нормандские лидеры"" призывали до конца 2019 года обеспечить прекращение огня."
8,8,2020-01-02 00:51:53,2020,1,2,"Экс-глава Национальной баскетбольной ассоциации Дэвид Стерн скончался в среду в возрасте 77 лет.\n\nСпортивный менеджер встал у руля NBA в 1984 году и возглавлял ее три десятка лет, сделав одной из самых богатых спортивных лиг в мире - третьей в Северной Америке после NFL (американский футбол) и MLB (бейсбол). В декабре Стерн был госпитализирован из-за кровоизлияния в мозг."
9,9,2020-01-02 03:52:11,2020,1,2,"МЧС хочет из зон чрезвычайных ситуаций при соответствующем решении властей.\n\nВ законопроекте, в частности, предусмотрены ""возможность принудительной эвакуации населения из зон ЧС"" и ""введение ответственности за несвоевременное представление информации о ЧС""."


# Gemini

In [9]:
import pandas as pd
import json
import re
import time
import math
import google.generativeai as genai
from IPython.display import display, HTML # Для display

# --- НАСТРОЙКИ ---
API_KEY_FILE = "config.py"
MODEL_NAME = "gemini-2.5-flash-preview-04-17"

TARGET_TOKENS_PER_BATCH = 15000 # Начнем с этого, как в успешном тесте
DAILY_REQUEST_LIMIT = 480
REQUEST_DELAY_SECONDS = 1.5 # Немного увеличим на всякий случай для preview
SAMPLE_SIZE_FOR_TESTING = 1000 # Количество строк для тестового запуска

# --- КОНЕЦ НАСТРОЕК ---

# Загрузка API ключа
try:
    from importlib import import_module
    config_module = import_module(API_KEY_FILE.replace('.py', ''))
    API_KEY = config_module.API_KEY
except (ImportError, AttributeError):
    print(f"Ошибка: Не удалось загрузить API_KEY из {API_KEY_FILE}.")
    exit()

# Конфигурация Google GenAI
try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Ошибка конфигурации Google GenAI: {e}")
    exit()

generation_config = {
  "temperature": 0.2, # Вернул к более низкому для точности
  "max_output_tokens": 65536,
  "response_mime_type": "application/json",
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

gemini_model = genai.GenerativeModel(
    model_name=MODEL_NAME,
    generation_config=generation_config,
    safety_settings=safety_settings
)

PROMPT_INSTRUCTION_TEMPLATE = """
Ты — редактор новостного агрегатора. Твоя задача - категоризировать список новостей.
Для каждой новости из списка предоставь JSON-объект.
Каждый такой объект должен содержать:
- "id": оригинальный идентификатор новости (целое число), который был предоставлен во входных данных.
- "multi_labels": Список (list) до 3 основных тем новости. Допустимые темы: Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости.
- "hier_label": Список (list) из двух элементов: [тема, подтема]. `тема` - это первая тема из `multi_labels`. `подтема` - это уточнение для этой основной темы (например, если тема "Политика", подтема может быть "Внешняя политика" или "Выборы").
Входные данные - это JSON-массив объектов, где каждый объект имеет "id" и "text".
Твой ответ ДОЛЖЕН БЫТЬ JSON-массивом, где каждый элемент массива - это JSON-объект с полями "id", "multi_labels", "hier_label" для соответствующей входной новости.
Убедись, что идентификаторы 'id' в твоем ответе точно соответствуют идентификаторам 'id' из входного списка новостей.
Не включай в ответ ничего, кроме этого JSON-массива.
Новости для обработки:
{news_json_payload}
Ответ (только JSON-массив):
"""

try:
    base_prompt_for_counting = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload="[]")
    TOKENS_FOR_BASE_PROMPT = gemini_model.count_tokens(base_prompt_for_counting).total_tokens
    print(f"Токены, занимаемые базовой инструкцией промпта: {TOKENS_FOR_BASE_PROMPT}")
except Exception as e:
    print(f"Не удалось посчитать токены для базового промпта: {e}. Используем примерное значение 350.")
    TOKENS_FOR_BASE_PROMPT = 350 # Запасное значение

def generate_categories_batch_gemini(news_batch_items):
    news_input_for_prompt = [{"id": item["id"], "text": item["text"]} for item in news_batch_items]
    news_input_json_string = json.dumps(news_input_for_prompt, ensure_ascii=False, indent=2)
    full_prompt = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload=news_input_json_string)
    
    # Подготавливаем "пустой" результат на случай ошибок, но с правильными ID
    empty_results_with_ids = [{"id": item["id"], "multi_labels": [], "hier_label": []} for item in news_batch_items]

    try:
        response = gemini_model.generate_content(full_prompt)
        
        # Отладочный вывод по ответу
        print(f"  DEBUG_API: Запрос для IDs с {news_batch_items[0]['id']} по {news_batch_items[-1]['id']} ({len(news_batch_items)} шт.)")
        if not response.parts:
            print(f"  ПРЕДУПРЕЖДЕНИЕ_API: Пустой ответ (response.parts пуст). Возможно, заблокировано.")
            if hasattr(response, 'prompt_feedback') and response.prompt_feedback:
                 print(f"    Причина блокировки: {response.prompt_feedback}")
            return empty_results_with_ids # Возвращаем пустые результаты с ID

        response_text = response.text
        print(f"  DEBUG_API: Длина response_text: {len(response_text)} символов.")
        if len(response_text) < 50: # Если ответ очень короткий, это подозрительно
             print(f"    Короткий response_text: '{response_text}'")
        
        if not response_text.strip(): # Если текст пустой или только пробелы
            print(f"  ПРЕДУПРЕЖДЕНИЕ_API: response_text пуст или состоит из пробелов.")
            return empty_results_with_ids

        try:
            parsed_json_list = json.loads(response_text)
            if not isinstance(parsed_json_list, list):
                print(f"  ОШИБКА_JSON: Ответ API не является JSON-массивом. Тип: {type(parsed_json_list)}. Ответ (начало): {response_text[:100]}...")
                return empty_results_with_ids
            
            # Валидация и сопоставление результатов
            # Создаем словарь из полученных результатов для быстрого доступа по id
            results_map_from_api = {item.get("id"): item for item in parsed_json_list if isinstance(item, dict)}
            
            # Формируем итоговый список для этого батча, гарантируя наличие всех запрошенных ID
            batch_final_results = []
            for requested_item in news_batch_items:
                req_id = requested_item["id"]
                if req_id in results_map_from_api:
                    api_item = results_map_from_api[req_id]
                    # Проверяем наличие и тип ключей
                    if isinstance(api_item.get("multi_labels"), list) and isinstance(api_item.get("hier_label"), list):
                        batch_final_results.append(api_item)
                    else:
                        print(f"  ПРЕДУПРЕЖДЕНИЕ_JSON: Некорректная структура категорий для id {req_id} в ответе API. Данные: {api_item}")
                        batch_final_results.append({"id": req_id, "multi_labels": [], "hier_label": []})
                else:
                    # Этого не должно происходить, если модель следует инструкциям, но лучше предусмотреть
                    print(f"  ПРЕДУПРЕЖДЕНИЕ_JSON: ID {req_id} из запроса не найден в ответе API.")
                    batch_final_results.append({"id": req_id, "multi_labels": [], "hier_label": []})
            return batch_final_results

        except json.JSONDecodeError as e:
            print(f"  ОШИБКА_JSON: Ошибка декодирования JSON: {e}\n    Текст ответа (начало): {response_text[:300]}...")
            return empty_results_with_ids

    except Exception as e:
        print(f"  КРИТИЧЕСКАЯ_ОШИБКА_API: При вызове API Gemini: {e}")
        return empty_results_with_ids


def process_dataframe_in_token_batches(df_input,
                                       target_tokens_total=TARGET_TOKENS_PER_BATCH,
                                       base_prompt_tokens=TOKENS_FOR_BASE_PROMPT,
                                       daily_request_limit=DAILY_REQUEST_LIMIT,
                                       request_delay=REQUEST_DELAY_SECONDS):
    df = df_input.copy()
    all_processed_results_flat = [] # Будет "плоским" списком словарей
    current_batch_items = []
    current_batch_estimated_tokens = base_prompt_tokens
    requests_count = 0

    df_results = df.copy() # Копия для модификации
    df_results["multi_labels"] = pd.Series([[] for _ in range(len(df_results))], index=df_results.index, dtype=object)
    df_results["hier_label"] = pd.Series([[] for _ in range(len(df_results))], index=df_results.index, dtype=object)


    print("Предварительный подсчет токенов для каждой новости...")
    news_with_tokens_info = []
    JSON_OVERHEAD_PER_ITEM_APPROX = 20 # Обновлено на основе ваших логов
    
    start_time_counting = time.time()
    for i, (idx, row) in enumerate(df.iterrows()):
        if (i + 1) % 250 == 0: print(f"  Подсчет токенов: обработано {i + 1}/{len(df)}...")
        
        news_text = str(row["text"])
        tokens_for_item = (len(news_text) // 4) + JSON_OVERHEAD_PER_ITEM_APPROX # Fallback: грубая оценка
        try:
            tokens_for_text_only = gemini_model.count_tokens(news_text).total_tokens
            tokens_for_item = tokens_for_text_only + JSON_OVERHEAD_PER_ITEM_APPROX
        except Exception as e:
            print(f"    Предупреждение: Ошибка подсчета токенов для id {idx} ('{str(e)[:50]}...'). Используется грубая оценка.")
        news_with_tokens_info.append({"id": idx, "text": news_text, "tokens": tokens_for_item})
    print(f"Предварительный подсчет токенов завершен. Заняло: {time.time() - start_time_counting:.2f}s")

    for news_item in news_with_tokens_info:
        if requests_count >= daily_request_limit:
            print(f"Достигнут дневной лимит запросов ({requests_count}). Обработка остановлена.")
            break

        item_tokens = news_item["tokens"]
        # Если текущий батч не пуст И добавление нового элемента превысит лимит
        if current_batch_items and (current_batch_estimated_tokens + item_tokens > target_tokens_total):
            print(f"Отправка батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
            batch_results_list = generate_categories_batch_gemini(current_batch_items) # должен вернуть список словарей
            requests_count += 1
            if isinstance(batch_results_list, list): # Убедимся, что это список
                 all_processed_results_flat.extend(batch_results_list)
            
            current_batch_items = []
            current_batch_estimated_tokens = base_prompt_tokens
            if requests_count < daily_request_limit and request_delay > 0: time.sleep(request_delay)

        # Если новость сама по себе (с базовым промптом) слишком велика
        if base_prompt_tokens + item_tokens > target_tokens_total:
            print(f"ПРЕДУПРЕЖДЕНИЕ: Новость id {news_item['id']} ({item_tokens} токенов) "
                  f"слишком велика даже для одиночной отправки с базовым промптом (лимит батча: {target_tokens_total}). "
                  "Новость будет пропущена.")
            # Добавляем "пустой" результат для этой новости, чтобы сохранить ID в all_processed_results_flat
            all_processed_results_flat.append({"id": news_item["id"], "multi_labels": [], "hier_label": []})
            continue # Переходим к следующей новости

        # Добавляем текущую новость в батч
        current_batch_items.append(news_item)
        current_batch_estimated_tokens += item_tokens

    # Отправляем последний оставшийся батч
    if current_batch_items and requests_count < daily_request_limit:
        print(f"Отправка последнего батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
        batch_results_list = generate_categories_batch_gemini(current_batch_items)
        requests_count += 1
        if isinstance(batch_results_list, list):
            all_processed_results_flat.extend(batch_results_list)

    print(f"Всего сделано запросов к API: {requests_count}")

    # Мерджим результаты в DataFrame
    if all_processed_results_flat:
        results_df = pd.DataFrame(all_processed_results_flat)
        
        print(f"DEBUG_MERGE: ---- Анализ перед мержем ----")
        print(f"DEBUG_MERGE: Всего элементов в results_df (из all_processed_results_flat): {len(results_df)}")
        if not results_df.empty:
            display(HTML("<b>results_df.head() перед мержем:</b>"))
            display(results_df.head())
            print(f"DEBUG_MERGE: dtypes results_df: \n{results_df.dtypes}")
            print(f"DEBUG_MERGE: dtypes df_results.index: {df_results.index.dtype}")
        
        if not results_df.empty and 'id' in results_df.columns:
            # Приведение типов ID для корректного мержа
            original_id_type = df_results.index.dtype
            results_df['id_original_type'] = results_df['id'] # Сохраним для отладки

            try:
                if pd.api.types.is_numeric_dtype(original_id_type):
                    results_df['id'] = pd.to_numeric(results_df['id'], errors='coerce')
                    results_df.dropna(subset=['id'], inplace=True)
                    if not results_df.empty:
                       results_df['id'] = results_df['id'].astype(original_id_type)
                elif original_id_type == 'object':
                     results_df['id'] = results_df['id'].astype(str)
                # Если другие типы индекса - можно добавить обработку
            except Exception as e:
                print(f"ПРЕДУПРЕЖДЕНИЕ_MERGE: Ошибка приведения типов ID: {e}. Мерж может быть неполным.")

            # Мерж с использованием индекса
            # Сначала установим индекс в results_df для эффективного обновления
            if not results_df.empty and 'id' in results_df.columns and results_df['id'].notna().all():
                try:
                    results_df = results_df.set_index('id')
                    # Обновляем df_results используя .loc и данные из results_df
                    # Это более pandas-way, чем итерация
                    updatable_idx = df_results.index.intersection(results_df.index)
                    
                    if not updatable_idx.empty:
                        print(f"DEBUG_MERGE: Найдено {len(updatable_idx)} совпадающих ID для обновления.")
                        # Прямое присвоение списков может вызывать проблемы, если не использовать .loc или .at
                        # Более безопасный способ - через apply или поэлементное присвоение,
                        # но для списков в ячейках object это должно работать.
                        # Однако, чтобы избежать ValueErrors, лучше итерировать по совпадающим индексам.
                        for idx_to_update in updatable_idx:
                            res_row = results_df.loc[idx_to_update]
                            df_results.at[idx_to_update, "multi_labels"] = res_row.get("multi_labels", [])
                            df_results.at[idx_to_update, "hier_label"] = res_row.get("hier_label", [])
                    else:
                        print("DEBUG_MERGE: Не найдено совпадающих ID для обновления.")
                except Exception as e:
                    print(f"ОШИБКА_MERGE: при установке индекса или обновлении df_results: {e}")
            else:
                print("ПРЕДУПРЕЖДЕНИЕ_MERGE: results_df пуст после приведения типов ID или содержит NaN в ID.")
        else:
            print("ПРЕДУПРЕЖДЕНИЕ_MERGE: DataFrame с результатами (results_df) пуст или не содержит колонку 'id'.")
    else:
        print("Не было получено результатов от API (all_processed_results_flat пуст).")
    return df_results

df_main = df.reset_index(drop=True).copy()


# --- Подготовка и запуск обработки ---
if len(df_main) > SAMPLE_SIZE_FOR_TESTING:
    df_to_process = df_main.head(SAMPLE_SIZE_FOR_TESTING).copy()
    print(f"Используется выборка из {SAMPLE_SIZE_FOR_TESTING} строк для обработки.")
else:
    df_to_process = df_main.copy()
    print(f"Используется полный DataFrame ({len(df_main)} строк) для обработки.")

CURRENT_RUN_TARGET_TOKENS = TARGET_TOKENS_PER_BATCH

print(f"\n--- Запуск обработки с целевым лимитом токенов на батч: {CURRENT_RUN_TARGET_TOKENS} ---")
start_processing_time = time.time()
df_processed = process_dataframe_in_token_batches(
    df_to_process,
    target_tokens_total=CURRENT_RUN_TARGET_TOKENS,
    base_prompt_tokens=TOKENS_FOR_BASE_PROMPT,
    daily_request_limit=DAILY_REQUEST_LIMIT,
    request_delay=REQUEST_DELAY_SECONDS
)
end_processing_time = time.time()
print(f"Общее время обработки: {end_processing_time - start_processing_time:.2f} секунд")

print("\n--- Обработанный DataFrame (первые 20 строк) ---")
display(df_processed.head(20))

filled_rows = df_processed[df_processed['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) > 0)].shape[0]
print(f"\nКоличество строк с заполненными категориями: {filled_rows} из {len(df_processed)}")

# Дополнительная проверка для строк, где метки остались пустыми
empty_label_rows = df_processed[df_processed['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
if not empty_label_rows.empty:
    print(f"\nСтроки, где метки остались пустыми ({len(empty_label_rows)} шт.):")
    display(empty_label_rows.head())

Токены, занимаемые базовой инструкцией промпта: 346
Используется выборка из 1000 строк для обработки.

--- Запуск обработки с целевым лимитом токенов на батч: 15000 ---
Предварительный подсчет токенов для каждой новости...
  Подсчет токенов: обработано 250/1000...
  Подсчет токенов: обработано 500/1000...
  Подсчет токенов: обработано 750/1000...
  Подсчет токенов: обработано 1000/1000...
Предварительный подсчет токенов завершен. Заняло: 84.96s
Отправка батча (198 новостей, ~14945 токенов). Запрос #1
  DEBUG_API: Запрос для IDs с 88230 по 79460 (198 шт.)
  DEBUG_API: Длина response_text: 36232 символов.
Отправка батча (199 новостей, ~14895 токенов). Запрос #2
  DEBUG_API: Запрос для IDs с 162445 по 138096 (199 шт.)
  DEBUG_API: Длина response_text: 36522 символов.
Отправка батча (197 новостей, ~14979 токенов). Запрос #3
  DEBUG_API: Запрос для IDs с 43741 по 24760 (197 шт.)
  DEBUG_API: Длина response_text: 34963 символов.
Отправка батча (199 новостей, ~14930 токенов). Запрос #4
  DEBU

,id,multi_labels,hier_label
0,88230,[Культура],"[Культура, Музыка]"
1,60779,"[Экономика, Политика, Международные отношения]","[Экономика, Энергетика]"
2,61944,"[Общество, Культура]","[Общество, События]"
3,127336,"[Политика, Общество, Международные отношения]","[Политика, Законодательство]"
4,71221,"[Спорт, Общество]","[Спорт, Футбол]"


DEBUG_MERGE: dtypes results_df: 
id               int64
multi_labels    object
hier_label      object
dtype: object
DEBUG_MERGE: dtypes df_results.index: int64
DEBUG_MERGE: Найдено 1000 совпадающих ID для обновления.
Общее время обработки: 1098.74 секунд

--- Обработанный DataFrame (первые 20 строк) ---


,index,date,year,month,day,text,multi_labels,hier_label
88230,88230,2022-03-28 15:59:56,2022,3,28,Музыкальный фестиваль Park Live перенесен на 2023,[Культура],"[Культура, Музыка]"
60779,60779,2021-08-20 17:59:04,2021,8,20,"Путин заявил, что РФ готова после 2024 г осуществлять транзит газа через Украину, объем и сроки контрактов будут зависеть от потребностей ЕС","[Экономика, Политика, Международные отношения]","[Экономика, Энергетика]"
61944,61944,2021-09-02 12:38:08,2021,9,2,"Итальянское Хотя в группе 4 хоббита, волшебник, эльф, гном и 2 людей, руководит ими и несет кольцо - вопреки традиции - не Фродо, а Сэм. Он рассказал РИА Новости, что в обычной жизни зовется Николас Джентиле и работает пекарем-кондитером.\n\nКто-то из участников экспедиции – банковский клерк, кто-то зарабатывает SMM, иной – рабочий, кто-то учится в университете. Кольцо они несут через полстраны.","[Общество, Культура]","[Общество, События]"
127336,127336,2023-09-05 16:41:44,2023,9,5,"Инициатива о запрете меховой индустрии за один день набрала в Финляндии необходимые 50 тысяч голосов, теперь ее рассмотрит парламент.\n\nВ прошлом месяце в стране выступили с идеей закрепить в конституции права животных, она также поступила в парламент и ждет голосования.","[Политика, Общество, Международные отношения]","[Политика, Законодательство]"
71221,71221,2021-11-29 11:47:05,2021,11,29,"13 заражений штаммом ""омикрон"" обнаружили в португальской футбольной команде ""Белененсеш"".\n\nВ субботу ""Белененсеш"" вышел на матч против ""Бенфики"" вдевятером. Во втором тайме (счет к тому моменту был уже 0:7 в пользу ""Бенфики"") на поле появились уже всего 7 игроков.\n\nПосле травмы одного из них матч остановили (вшестером уже нельзя играть по правилам).","[Спорт, Общество]","[Спорт, Футбол]"
52319,52319,2021-06-09 09:51:01,2021,6,9,"Комментатор телеканал ""Матч ТВ"" Нобель Арустамян не получил аккредитацию на Евро-2020. Для работы на турнире, который пройдет в 11 городах разных стран, журналисты должны получить одобрение всех оргкомитетов стран-хозяек. Такого одобрения нет от оргкомитета Баку.\n\nВ Ассоциации футбольных федераций Азербайджана этот отказ связали с тем, что журналист «неоднократно совершал незаконные поездки на оккупированные азербайджанские земли» без разрешения властей страны, «проявив неуважение к территориальной целостности».\n\nГенеральный продюсер ""Матч ТВ"" Тина Канделаки назвала ситуацию ""нарушением духа международного спортивного движения"", отметив, что телеканал делает все возможное, чтобы разрешить эту ситуацию.","[Спорт, Политика, Международные отношения]","[Спорт, Футбол]"
98448,98448,2022-07-16 09:56:04,2022,7,16,"Шойгу проинспектировал российские группировки войск ""Юг"" и ""Центр"", выполняющие задачи специальной военной операции на Украине","[Политика, Международные отношения, Происшествия]","[Политика, Военные действия]"
35758,35758,2021-01-07 00:56:08,2021,1,7,"Пенс поставил на заставку своего личного Twitter фото Байдена с Харрис, но от Трампа не отписался","[Политика, Международные отношения]","[Политика, Внутренняя политика США]"
16068,16068,2020-06-03 13:21:13,2020,6,3,"Кремль надеется, что компетентные ведомства примут меры к ""Здесь можно выразить только крайнее сожаление в связи с тем, что такая игра доступна на территории нашей страны. И доступна тем более сейчас, в год, когда мы отмечаем 75-летие Победы. Потому что, конечно, эта игра, с нашей точки зрения, не что иное, как просто пропаганда нацизма. Что недопустимо""","[Политика, Общество, Культура]","[Политика, Внутренняя политика]"
62545,62545,2021-09-08 10:58:13,2021,9,8,Горячая линия по взрыву в Ногинске +7 495 542 21 41,"[Происшествия, Региональные новости]","[Происшествия, Чрезвычайные ситуации]"



Количество строк с заполненными категориями: 1000 из 1000


In [13]:
df_processed.to_csv('1k.csv', index=False)

In [11]:
# from google import genai

# import config  # Импортируем файл конфигурации

# client = genai.Client(api_key=config.API_KEY)  # Используем API ключ из конфигурации

# response = client.models.generate_content(
#     model="gemini-2.0-flash",
#     contents="Здарова бандит",
# )

# print(response.text)